## Soynlp 학습기반 토크나이저
<hr>

- 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저
- 비지도 학습으로 단어 토큰화 ==> 데이터에 자주 등장하는 단어들을 단어로 분석
- 내부적으로 단어 점수 표로 인식

In [3]:
# !pip install soynlp

In [14]:
from konlpy.tag import Mecab, Okt

tokenizer = Okt()
sentence = '에이비식스 이대휘 1월 최애돌 기부 요정 입니다.'

# 형태소 분석 시 매개변수 stem = True 설정
print(tokenizer.pos('에이비식스 이대휘 1월 최애돌 기부 요정 입니다.', stem=True))
print(tokenizer.pos('에이비식스이대휘1월최애돌기부요정입니다.'))



[('에이', 'Noun'), ('비식스', 'Noun'), ('이대', 'Modifier'), ('휘', 'Noun'), ('1월', 'Number'), ('최애', 'Noun'), ('돌', 'Noun'), ('기부', 'Noun'), ('요정', 'Noun'), ('이다', 'Adjective'), ('.', 'Punctuation')]
[('에이', 'Noun'), ('비식스', 'Noun'), ('이대', 'Modifier'), ('휘', 'Noun'), ('1월', 'Number'), ('최애돌기부요정입니다', 'Foreign'), ('.', 'Punctuation')]


### 토크나이저 사용시 미리 말뭉치 데이터셋으로 학습 후 사용 가능


In [17]:
filename = 'text_data.txt'
basedir = '../DATA/'

# 한개로 통합된 문서 데이터 정리
from soynlp import DoublespaceLineCorpus
# 단어 추출
from soynlp.word import WordExtractor

corpus = DoublespaceLineCorpus(basedir + filename)
print(f"훈련 데이터 문서 : {len(corpus)}개")


훈련 데이터 문서 : 30091개


In [18]:

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()


training was done. used memory 1.277 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [19]:
for idx, key in enumerate(word_score_table.keys()):
    print(f"{idx+1}. {key} : {word_score_table[key]}")
    if idx > 30: break

1. 춘 : Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=3.5202186749788362, right_branching_entropy=3.6047001222868227, left_accessor_variety=60, right_accessor_variety=132, leftside_frequency=549, rightside_frequency=818)
2. 싫 : Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=2.853861143073829, right_branching_entropy=1.5625414514665827, left_accessor_variety=31, right_accessor_variety=8, leftside_frequency=280, rightside_frequency=0)
3. 딸 : Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=3.245875269275488, right_branching_entropy=3.2099383945584727, left_accessor_variety=76, right_accessor_variety=65, leftside_frequency=574, rightside_frequency=32)
4. 혀 : Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=1.6934449658569768, right_branching_entropy=3.959341704691131, left_accessor_variety=7, right_accessor_variety=113, leftside_frequency=35, rightside_frequency=1061)
5. 리 : Scores(cohesion_forward

In [29]:
word_score_table['바다를'].cohesion_forward, word_score_table['바다에'].cohesion_forward

(0.07716779358040307, 0.11518621707955429)

In [33]:
from soynlp.tokenizer import LTokenizer

# 토큰으로 쪼개기 위한 L토큰 기준값
scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize('국제사회와 우리의 노력들로 범죄를 척결하자', flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [35]:
### 최대 점수 토크나이저
# 띄어쓰기가 되지 않는 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저
# 띄어쓰기가 되어 있지 않는 문장을 넣어서 점수를 통해 토큰화 된 결과

from soynlp.tokenizer import MaxScoreTokenizer

maxscore_token = MaxScoreTokenizer(scores=scores)
maxscore_token.tokenize('국제사회와우리의노력들로범죄를척결하자', flatten=False)

[[('국제사회', 0, 4, 0.20075093164820862, 4),
  ('와', 4, 5, 0.0, 1),
  ('우리', 5, 7, 0.4698248708580068, 2),
  ('의', 7, 8, 0.0, 1),
  ('노력', 8, 10, 0.17947431781701445, 2),
  ('들로', 10, 12, 0.0, 2),
  ('범죄', 12, 14, 0.29525483304042177, 2),
  ('를', 14, 15, 0.0, 1),
  ('척결', 15, 17, 0.1326530612244898, 2),
  ('하자', 17, 19, 0.0, 2)]]

In [49]:
## soynlp 를 이용한 반복되는 문자 정제
# ㅋㅋ, ㅎㅎ 등의 이모티콘의 경우 불필요하게 연속되는 경우가 많음
# 반복되는 것은 하나로 정규화

from soynlp.normalizer import *

import string
print(string.punctuation)

print(emoticon_normalize('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ큐ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ'), emoticon_normalize('영화ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ감동적^^^^^^'))




!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
ㅋㅋㅠㅠ 영화ㅠㅠ감동적^^^^^^


In [ ]:
# !pip install customized_konlpy

In [65]:
from ckonlpy.tag import Twitter


twitter = Twitter()
twitter.morphs('은경이는 사무실로 갔습니다.')

['은', '경이', '는', '사무실', '로', '갔습니다', '.']

In [66]:
twitter.add_dictionary('은경이', 'Noun')
twitter.morphs('은경이는 사무실로 갔습니다.')


['은경이', '는', '사무실', '로', '갔습니다', '.']

In [71]:

c_mecab = Mecab()
c_mecab.morphs('은경이는 사무실로 갔습니다.')


['은경', '이', '는', '사무실', '로', '갔', '습니다', '.']